# Линейный SVM "своими руками"

## Генерируем обучающую и тестовую выборку для экспериментов

In [1]:
from sklearn.model_selection import train_test_split
from sklearn import datasets

X, y = datasets.make_classification(
    n_samples=10000, n_features=20, 
    n_classes=2, n_informative=20, 
    n_redundant=0,
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,
    random_state=42
)

print(len(X), len(y))
print(len(X_train))

10000 10000
8000


## Пишем свой класс для SVM

In [2]:
import numpy as np
from random import randint
import random


np.random.seed(42)
random.seed(42)


class MySVM(object):
    def __init__(self, C=10000):
        self.C = C # regularization constant

    # f(x) = <w,x> + w_0
    def f(self, x):
        return np.dot(self.w, x) + self.w0

    # a(x) = [f(x) > 0]
    def a(self, x):
        return 1 if self.f(x) > 0 else 0
    
    # predicting answers for X_test
    def predict(self, X_test):
        return np.array([model.a(x) for x in X_test])

    # l2-regularizator
    def reg(self):
        return 1.0 * sum(self.w ** 2) / (2.0 * self.C)

    # l2-regularizator derivative
    def der_reg(self):
        return self.w / self.C;

    # hinge loss
    def loss(self, x, answer):
        return max([0, 1 - answer * self.f(x)])

    # hinge loss derivative
    def der_loss(self, x, answer):
        return -1.0 if 1 - answer * self.f(x) > 0 else 0.0;

    # fitting w and w_0 with SGD
    def fit(self, X_train, y_train):
        dim = len(X_train[0])
        self.w = np.random.rand(dim) # initial value for w
        self.w0 = np.random.randn() # initial value for w_0
        
        # 10000 steps is OK for this example
        # another variant is to continue iterations while error is still decreasing
        for k in range(10000):  
            
            # random example choise
            rand_index = randint(0, len(X_train) - 1) # generating random index
            x = X_train[rand_index]
            y = y_train[rand_index]

            # simple heuristic for step size
            step = 0.5 * 0.9 ** k

            # w update
            self.w0 = self.w0 - step * y * self.der_loss(x,y);         
            # w_0 update
            self.w = self.w - step * (y * x * self.der_loss(x,y) + self.der_reg());

## Пробуем обучить наш классификатор и посмотреть на качество на тесте

In [3]:
model = MySVM()
model.fit(X_train, y_train)
print(model.w, model.w0)

[-0.40728934  2.00502273  1.4527374   2.23584225  1.2716151   1.56342388
 -1.86684573 -0.33232009  0.04373314 -2.05910313 -0.94063898 -0.6921571
  1.5003582  -2.44346304 -1.63989848 -0.05618576  0.34564959  4.76830967
  2.31519674  0.31211818] 0.406819600109


In [4]:
predictions = model.predict(X_test)

In [5]:
print(predictions)

[0 1 0 ..., 1 1 1]


In [6]:
print(y_test, len(y_test), sum(y_test))

[1 0 1 ..., 1 0 1] 2000 991


In [7]:
print(len(predictions), sum(predictions))

2000 1231


In [8]:
print(sum(predictions == y_test) / float(len(y_test)))

0.61


# LinearSVC

In [9]:
from sklearn.svm import LinearSVC
model = LinearSVC()

In [10]:
model.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [11]:
predictions = model.predict(X_test)

In [12]:
print(sum(predictions == y_test) / float(len(y_test)))

0.7985


## Задания:

### - Допишите недостающие функции в MySVM (производные и обновление весов)

### - Сравните качество с sklearn LinearSVC